# Data Exploration

This notebook is more or less a sandbox made for me to understand and decide how to prepare the dataset and what target to select.

### Constants and imports

### Loading common dataset

### Checking technique values

### Joining similar values

### Preparing the final dataframe and dividing into train/test data

### Saving the .csv file